# 랭체인을 활용한 검색 증강 생성(Retrieval Augmented Generation; RAG)

이 노트북 예제에서는 아마존사의 주주서한 데이터를 활용한 RAG를 실행하여 기본적인 Q&A를 실행 해보겠습니다.

이 노트북은 특별한 CPU/GPU 사양이 필요없으며, '파이썬 3 데이터사이언스 3.0' 커널을 통해 빌드되었습니다.

## 의존성

이 예제를 위한 의존성 설치:
- 랭체인: RAG 워크플로를 오케스트레이션하기 위한 프레임워크
- FAISS: 문서 임베딩을 저장하기 위한 인메모리 벡터 데이터베이스
- PyPDF: PDF 문서를 처리하기 위한 파이썬 라이브러리

In [2]:
%pip install langchain==0.0.309 --quiet --root-user-action=ignore
%pip install faiss-cpu==1.7.4 --quiet --root-user-action=ignore
%pip install pypdf==3.15.1 --quiet --root-user-action=ignore


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 샘플 데이터 수집과 처리

다음으로, 이 예제를 위한 샘플 데이터를 가져옵니다. 이 섹션에서는 매년 아마존사의 "연간 리뷰 (Year in Review)"로 제공되는 공개적으로 이용 가능한 아마존사 주주서한을 다운로드합니다.

PDF 파일을 로컬로 다운로드하여 이 노트북의 `data` 디렉토리에 저장합니다.

In [3]:
!mkdir -p ./data

from urllib.request import urlretrieve
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf'
]

filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
    'AMZN-2021-Shareholder-Letter.pdf',
    'AMZN-2020-Shareholder-Letter.pdf',
    'AMZN-2019-Shareholder-Letter.pdf'
]

metadata = [
    dict(year=2022, source=filenames[0]),
    dict(year=2021, source=filenames[1]),
    dict(year=2020, source=filenames[2]),
    dict(year=2019, source=filenames[3])]

data_root = "./data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

아마존사의 독특한 문화로, CEO는 항상 현재의 주주들에게 보내는 편지에 1997년의 원본 편지를 첨부합니다. 처리해야 할 양을 줄이고 해당 연도에 대한 편향을 줄이며 출력 품질을 개선하기 위해, PyPDF를 사용하여 각 파일에서 해당 페이지를 제거한 후 원본 파일에 덮어쓰겠습니다.

In [4]:
from pypdf import PdfReader, PdfWriter
import glob

local_pdfs = glob.glob(data_root + '*.pdf')

for local_pdf in local_pdfs:
    pdf_reader = PdfReader(local_pdf)
    pdf_writer = PdfWriter()
    for pagenum in range(len(pdf_reader.pages)-3):
        page = pdf_reader.pages[pagenum]
        pdf_writer.add_page(page)

    with open(local_pdf, 'wb') as new_file:
        new_file.seek(0)
        pdf_writer.write(new_file)
        new_file.truncate()


이제 작업할 수 있는 정리된 PDF 파일이 준비되었으므로, RAG 워크플로 상에서 LLM에 가장 관련 있는 섹션을 제공할 수 있도록 이를 적절한 크기로 분할해야 합니다. 여기에서는 모든 문서를 반복 처리하여 512자 크기로 나누고, 100자씩 곂쳐서 분할합니다.

`chunk_size`는 임베딩되어 벡터 데이터베이스에 저장될 청크의 크기를 나타냅니다. 
'chunk_size' dictates the size of the documents that will be embedded and stored in the vector database.

`chunk_overlap`은 청크를 분리할때 이전 청크에서 중복해서 사용된 테스트의 양을 나타냅니다. 이는 청크간 맥락을 유지할 수 있도록 해줍니다.

`RecursiveCharacterTextSplitter`는 `["\n\n", "\n", " ", ""]`와 같은 구분자를 사용하여 원하는 청크 크기가 만들어질때까지 반복적으로 분할하려고 시도합니다. 이러한 시도는 문단/문장/단어를 유지함으로써 더 나은 의도분석을 가능하게 합니다.

In [5]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

documents = []

for idx, file in enumerate(filenames):
    loader = PyPDFLoader(data_root + file)
    document = loader.load()
    for document_fragment in document:
        document_fragment.metadata = metadata[idx]
        
    documents += document

# - 테스트 결과 PDF 데이터셋에서 텍스트 분리가 잘됨
text_splitter = RecursiveCharacterTextSplitter(
    # 내용을 보기위해 매우 작은 청크사이즈로 설정
    chunk_size = 512,
    chunk_overlap  = 100,
)

docs = text_splitter.split_documents(documents)

print(f'# of Document Pages {len(documents)}')
print(f'# of Document Chunks: {len(docs)}')

# of Document Pages 25
# of Document Chunks: 299


## 임베딩 모델 배포

다음 섹션에서는 ML모델 세트를 배포하게 되는데, 하나는 임베딩을 위한것이고, 다른 하나는 언어 생성을 위한 LLM입니다. 이 예제는 SageMaker Studio 내에서 작업하고 있다고 가정하며, 직접 배포하거나 SageMaker Jumpstart를 통해 배포할 수 있습니다.

이 예제에서는 임베딩 모델로 `All MiniLM L6 v2`를 사용하고, 언어 생성을 위한 LLM으로 `LLaMa-2-7B-chat`을 사용할 것입니다.e generation. 

__노트:__ 다른 옵션을 선택할 경우, 선택한 모델과 임베딩 및 LLM을 맞추기 위해 이후 섹션의 `transform_input`과 `transform_output` 함수를 조정해야 할 수도 있습니다.

점프스타트로 모델을 배포하는 방법에 대해서는 [세이지메이커 점프스타트 문서](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-jumpstart.html)를 참고하세요.

만약 이미 배포된 임베딩 엔드포인트가 있다면 다음 셀은 넘어가도 되며, `embedding_model_endpoint_name` 값만 엔드포인트에 맞춰서 수정하세요.

__노트: 다음 셀을 실행하면 세이지메이커 엔드포인트를 배포하게 됩니다. 과도한 과금을 피하기 위해서는 테스트 후 엔드포인트를 삭제해야 합니다. 이 노트북 마지막에 있는 클린업 단계를 살펴 보세요.__

In [6]:
from sagemaker.jumpstart.model import JumpStartModel

embedding_model_id, embedding_model_version = "huggingface-textembedding-all-MiniLM-L6-v2", "*"
model = JumpStartModel(model_id=embedding_model_id, model_version=embedding_model_version)
embedding_predictor = model.deploy()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


Using model 'huggingface-textembedding-all-MiniLM-L6-v2' with wildcard version identifier '*'. You can pin to version '1.0.0' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


---------!

In [7]:
# ARN이 아니라 모델 엔드포인트 이름임
embedding_model_endpoint_name = embedding_predictor.endpoint_name
embedding_model_endpoint_name

'hf-textembedding-all-minilm-l6-v2-2023-12-26-20-58-21-407'

세이지메이커 모델 엔드포인트를 사용하려면 자격증명이 필요합니다. 이번 섹션에서는 SageMaker Studio 세션으로 가정하겠습니다.

In [8]:
import boto3
aws_region = boto3.Session().region_name

## 벡터 데이터베이스 생성과 데이터 구축

다음으로 입력된 문서에 대한 임베딩을 처리하는 방법을 설정해야 합니다.

제공된 CustomEmbeddingsContentHandler 클래스에는 임베딩 모델로 들어가고 나오는 데이터를 처리하기 위한 transform_input과 transform_output 함수가 포함되어 있습니다.

콘텐츠 핸들러를 정의한 후에는 랭체인의 SageMakerEndpointEmbeddings 클래스를 사용하여, 호스팅된 임베딩 모델과 입력/출력을 처리할 적절한 콘텐츠 핸들러로 임베딩 객체를 생성하게 됩니다.

In [9]:
from typing import Dict, List
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler
import json


class CustomEmbeddingsContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"text_inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["embedding"]


embeddings_content_handler = CustomEmbeddingsContentHandler()


embeddings = SagemakerEndpointEmbeddings(
    endpoint_name=embedding_model_endpoint_name,
    region_name=aws_region,
    content_handler=embeddings_content_handler,
)

임베딩이 준비되면 다음 단계는 문서 청크를 벡터로 만들어 어딘가에 저장하는 것입니다. 이 예제는 FAISS 인메모리 벡터 데이터베이스를 사용하지만 다른 여러가지 다른 옵션들도 있습니다.

In [10]:
from langchain.schema import Document
from langchain.vectorstores import FAISS

In [11]:
db = FAISS.from_documents(docs, embeddings)

## 벡터 쿼리 실행

이제 벡터 데이터베이스가 만들어졌으니, 관련된 문서 청크를 찾기위해 쿼리를 실행할 수 있습니다. 

자료에 대한 단순한 쿼리부터 시작해봅시다.

In [12]:
query = "How has AWS evolved?"

`similarity_search_with_score` API에서 받은 결과는 점수의 오름차순으로 정렬됩니다. 점수값은 각 결과의 [L-squared (or L2)](https://en.wikipedia.org/wiki/Lp_space) 거리로 표현됩니다. 더 작은 점수가 더 좋은, 즉 벡터간 거리가 짧은 것입니다.

In [13]:
results_with_scores = db.similarity_search_with_score(query)
for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}\nMetadata: {doc.metadata}\nScore: {score}\n\n")

Content: done innovating here,and this long-term investment should prove fruitful for both customers and AWS. AWS is still in the earlystages of its evolution, and has a chance for unusual growth in the next decade.
Metadata: {'year': 2022, 'source': 'AMZN-2022-Shareholder-Letter.pdf'}
Score: 0.5685306191444397


Content: customers, AWS continues to deliver new capabilities rapidly (over 3,300 new features and services launchedin 2022), and invest in long-term inventions that change what’s possible.
Metadata: {'year': 2022, 'source': 'AMZN-2022-Shareholder-Letter.pdf'}
Score: 0.7789842486381531


Content: We had a head start on potential competitors;and if anything, we wanted to accelerate our pace of innovation. We made the long-term decision tocontinue investing in AWS. Fifteen years later, AWS is now an $85B annual revenue run rate business, withstrong profitability, that has transformed how customers from start-ups to multinational companies to publicsector organizations manage the

In [14]:
filter={"year": 2022}

results_with_scores = db.similarity_search_with_score(query,
  filter=filter)

for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}\nMetadata: {doc.metadata}\nScore: {score}\n\n")


Content: done innovating here,and this long-term investment should prove fruitful for both customers and AWS. AWS is still in the earlystages of its evolution, and has a chance for unusual growth in the next decade.
Metadata: {'year': 2022, 'source': 'AMZN-2022-Shareholder-Letter.pdf'}
Score: 0.5685306191444397


Content: customers, AWS continues to deliver new capabilities rapidly (over 3,300 new features and services launchedin 2022), and invest in long-term inventions that change what’s possible.
Metadata: {'year': 2022, 'source': 'AMZN-2022-Shareholder-Letter.pdf'}
Score: 0.7789842486381531


Content: We had a head start on potential competitors;and if anything, we wanted to accelerate our pace of innovation. We made the long-term decision tocontinue investing in AWS. Fifteen years later, AWS is now an $85B annual revenue run rate business, withstrong profitability, that has transformed how customers from start-ups to multinational companies to publicsector organizations manage the

## 프롬프트 작성

벡터 데이터베이스에서 결과를 얻었지만, 현재는 원본 문서의 청크일 뿐이며, 그 중 일부는 원래 쿼리에 대한 답변으로 제공하고자 하는 정보를 포함하지 않을 수도 있습니다.

적절한 답변을 생성하기 위해, 원래 질문과 벡터 데이터베이스에서 가져온 관련 청크를 사용하여 언어 생성 모델로부터 새로운 응답을 생성하는 프롬프트 템플릿을 활용할 것입니다.

랭체인은 프롬프트 템플릿을 더 쉽게 생성할 수 있는 기능을 제공합니다. 아래 템플릿은 LLaMa-2-chat에 대한 특정 마크업을 포함하고 있으며, 템플릿을 채우기위해 제공하는 `{context}`와 `{question}` 위치 표시자를 포함하고 있습니다.

In [15]:
from langchain.prompts import PromptTemplate

prompt_template = """
<s>[INST] <<SYS>>
Use the context provided to answer the question at the end. If you dont know the answer just say that you don't know, don't try to make up an answer.
<</SYS>>

Context:
----------------
{context}
----------------

Question: {question} [/INST]
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

## LLM 준비

다음 단계는 임베딩 모델을 위해 앞서 수행한 것과 유사한 작업이지만 이번에는 LLM에 대한 것입니다.

QAContentHandler 클래스에는 LLM의 입력과 출력을 처리하는 `transform_input`과 `transform_output` 함수가 있습니다.

In [16]:
from typing import Dict

from langchain import PromptTemplate, SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
import json


class QAContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        input_str = json.dumps(
            {"inputs" : [
                [
                    {
                        "role" : "system",
                        "content" : ""
                    },
                    {
                        "role" : "user",
                        "content" : prompt
                    }
                ]],
                "parameters" : {**model_kwargs}
            })
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generation"]["content"]

qa_content_handler = QAContentHandler()

이제 언어 생성 LLM을 위한 세이지메이커 엔드포인트를 배포할 것입니다. 이후, 해당 엔드포인트를 가리키는 객체를 생성하고, 엔드포인트와 모델에 추론 매개변수를 제공하게 됩니다.

이미 배포된 LLM 엔드포인트가 있다면, 다음 셀은 넘어가도 되며 `llm_model_endpoint_name` 값을 엔드포인트와 맞는 값으로 수정해주세요.

__Note: 다음 셀을 실행하면 세이지메이커 엔드포인트를 배포하는데 수 분이 걸립니다. 과도한 과금을 피하기 위해서는 테스트 후 엔드포인트를 삭제해야 합니다. 이 노트북 마지막에 있는 클린업 단계를 살펴 보세요.__

In [ ]:
llm_model_id, llm_model_version = "meta-textgeneration-llama-2-7b-f", "2.*"
llm_model = JumpStartModel(model_id=llm_model_id, model_version=llm_model_version)
llm_predictor = llm_model.deploy()

In [18]:
# ARN이 아니라 모델 엔드포인트 이름임
llm_model_endpoint_name = llm_predictor.endpoint_name
llm_model_endpoint_name

'meta-textgeneration-llama-2-7b-f-2023-12-26-21-03-42-236'

In [19]:
llm = SagemakerEndpoint(
        endpoint_name=llm_model_endpoint_name,
        region_name=aws_region,
        model_kwargs={"max_new_tokens": 1000, "top_p": 0.9, "temperature": 1e-11},
        endpoint_kwargs={"CustomAttributes": 'accept_eula=true'},
        content_handler=qa_content_handler
    )

기본적인 응답을 받기위해서는 어떠한 맥락 정보도 없이 LLM 객체를 직접 호출할 수도 있습니다. `AWS가 어떻게 발전해 왔는가?`라는 질문에 대한 답변에 AWS가 내부적인 관점에서 어떻게 발전했는지 보다는 __무엇__ 을 했는지에 대한 내용이 주를 이룬다는 것을 알 수 있습니다. 이는 LLM을 훈련한 데이터 집합에 인터넷에서 수집된 많은 기사들이 포함되어 있기 때문일 가능성이 큽니다.

이 답변이 결코 나쁜 것은 아니지만, 기대했던 답변과는 다를 수 있습니다.

그럼 콘텍스트가 어떻게 답변을 개선시킬 수 있는지 보겠습니다.

In [20]:
query = "How has AWS evolved?"
llm.predict(query)

" AWS (Amazon Web Services) has evolved significantly since its launch in 2006. Here are some key milestones and developments in AWS's evolution:\n\n1. 2006: Amazon Web Services (AWS) is launched as a separate business unit within Amazon, offering a limited set of cloud computing services, including Elastic Compute Cloud (EC2), Simple Storage Service (S3), and Simple Queue Service (SQS).\n2. 2008: AWS introduces its first virtual private cloud (VPC), allowing customers to launch AWS resources in a dedicated virtual network.\n3. 2010: AWS launches its first data center outside of the United States, in Ireland.\n4. 2011: AWS introduces the Elastic Block Store (EBS), providing block-level storage volumes for EC2 instances.\n5. 2012: AWS launches its CloudFormation service, allowing customers to define and manage AWS infrastructure using templates.\n6. 2013: AWS introduces the Amazon Elastic Container Service (ECS), providing a highly scalable, high-performance container orchestration serv

생성한 LLM 엔드포인트 객체를 통해 첫번째 체인을 생성할 준비가 되었습니다.

이 체인은 랭체인의 RetrievalQA 체인을 사용한 간단한 예제입니다: 
- 입력으로 쿼리를 받습니다.
- 쿼리 임베딩을 생성합니다.
- 쿼리 임베딩과 관련된 문서 청크를 벡터 데이터베이스에서 찾습니다.
- 콘텍스트와 원본 쿼리를 프롬프트 템플릿에 입력합니다.
- 완성된 프롬프트로 LLM을 호출합니다.
- LLM 결과를 리턴합니다.

[`stuff` 체인 유형](https://python.langchain.com/docs/modules/chains/document/stuff)은 단순히 컨텍스트 문서를 프롬프트에 삽입합니다.

`return_source_documents`를 `True`로 설정함으로써, LLM 응답에 벡터 데이터베이스에서 가져온 문서 청크도 포함하어, 컨텍스트가 어디에서 왔는지 보여줍니다.

In [21]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type='stuff',
    retriever=db.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

이제 체인이 준비되었으니 쿼리를 전달하여 원천 문서를 기반으로 답변이 생성되도록 할 수 있습니다. 

몇가지 예제가 제공됩니다.

In [22]:
query = "How has AWS evolved?"
result = qa_chain({"query": query})
print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')

Query: How has AWS evolved?

Result:  Based on the provided context, AWS has evolved in the following ways:

1. Rapid innovation: AWS continues to deliver new capabilities rapidly, launching over 3,300 new features and services in 2022 alone.
2. Long-term investment: AWS has made a long-term decision to continue investing in its infrastructure, even during challenging times such as the 2008-2009 recession.
3. Expansion of services: AWS has expanded its offerings beyond just computing and storage, now providing a wide range of services including analytics, machine learning, and security.
4. Increased profitability: Despite continued investment in innovation, AWS has achieved strong profitability, with an $85B annual revenue run rate business.
5. Shift to cloud adoption: The pandemic has accelerated the shift to cloud adoption, with many companies deciding to move their technology infrastructure to the cloud. This has helped re-accelerate AWS's revenue growth to 37% YoY in 2021.

Overall

In [23]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type='stuff',
    retriever=db.as_retriever(
        search_type="mmr", # 최대 한계 관련성 (Maximum Marginal Relevance; MMR)
        search_kwargs={"k": 3, "lambda_mult": 0.1}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

다시 체인이 준비되었으니 쿼리를 전달하여 원천 문서를 기반으로 답변이 생성되도록 할 수 있습니다. 

몇가지 예제가 제공됩니다.

In [24]:
query = "How has AWS evolved?"
result = qa_chain({"query": query})
print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')

Query: How has AWS evolved?

Result:  Based on the context provided, AWS has evolved in the following ways:

1. Innovation: AWS has continued to innovate and invest in new technologies and services to meet the changing needs of its customers.
2. Growth: AWS is expected to experience unusual growth in the next decade, particularly in the areas of virtual classrooms and government efforts to combat the pandemic.
3. Efficiency: AWS is more efficient than traditional in-house data centers due to its institutional advantages and the transition to virtual classrooms and government use.

I don't know the answer to the question "What are the specific areas of innovation for AWS?" as the context does not provide that information.

Context Documents: 
page_content='done innovating here,and this long-term investment should prove fruitful for both customers and AWS. AWS is still in the earlystages of its evolution, and has a chance for unusual growth in the next decade.' metadata={'year': 2022, 's

In [25]:
query = "Why is Amazon successful?"
result = qa_chain({"query": query})
print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')

Query: Why is Amazon successful?

Result:  Based on the provided context, Amazon's success can be attributed to several factors:

1. Early Mover Advantage: Amazon has been iterating and remaking its fulfillment capabilities for nearly two decades, giving it a significant head start over its competitors. This early mover advantage has allowed Amazon to develop a robust and efficient logistics network, which has been critical in its success.
2. Customer Obsession: Amazon is divinely discontented with customer experiences, whether they are its own or not. The company is constantly experimenting and inventing to make customers' experiences better. This customer-centric approach has helped Amazon build a loyal customer base and differentiate itself from its competitors.
3. Innovation: Amazon is known for its innovative approach to business. The company is always looking for new ways to improve its operations and customer experiences. This innovative spirit has allowed Amazon to stay ahead o

In [26]:
query = "What business challenges has Amazon experienced?"
result = qa_chain({"query": query})
print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')

Query: What business challenges has Amazon experienced?

Result:  Based on the provided context, Amazon has experienced the following business challenges:

1. Rising cost to serve in their Stores fulfillment network (i.e. the cost to get a product from Amazon to a customer).
2. Unusual number of simultaneous challenges this past year.
3. Operating in large, dynamic, global market segments with many capable and well-funded competitors.

It is important to note that the context only provides information about the challenges Amazon has experienced, and does not provide information about the company's overall performance or success.

Context Documents: 
page_content='shareholders, and employees.\nWhile there were an unusual number of simultaneous challenges this past year, the reality is that if you\noperate in large, dynamic, global market segments with many capable and well-funded competitors (theconditions in which Amazon operates all of its businesses), conditions rarely stay stagnant 

# 클린업

`delete_endpoint` 호출의 커멘트를 해제하여 생성한 리소스들을 제거하세요.

In [ ]:
# sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

# #임베딩 엔드포인트 삭제
# sagemaker_client.delete_endpoint(EndpointName=embedding_model_endpoint_name)

# #LLM 엔드포인트 삭제
# sagemaker_client.delete_endpoint(EndpointName=llm_model_endpoint_name)